In [1]:
from data_loaders import ModularCharatersDataLoader
from models import DDPMNextTokenV2
from datasets import load_dataset



train_dataloader = ModularCharatersDataLoader.get_modular_char_dataloader(dataset_name='QLeca/modular_characters_small',
                                                                            split='train',
                                                                            image_size=128,
                                                                            batch_size=16,
                                                                            shuffle=True)
dataset = load_dataset('QLeca/modular_characters_small', 
                       split='train')

prompts = dataset['prompt']
vocab = list(dict.fromkeys(prompts))
vocab = sorted(vocab)
vocab = dict(zip(vocab, range(len(vocab))))
pipeline = DDPMNextTokenV2.DDPMNextTokenV2Pipeline()
pipeline.set_class_vocabulary(vocab.keys())
  

wandb: Currently logged in as: quentinlca (quentinlca-perso) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
d:\Programmes\miniconda3\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
d:\quent\Documents\Copenhague\Stage\Recherche\diffusion_research\layer_diffuse\training_outputs/DDPMNextTokenV2 is already a clone of https://huggingface.co/QLeca/DDPMNextTokenV2. Make sure you pull the latest changes with `repo.git_pull()`.
Checked out main from main.
Your branch is up to date with 'origin/main'.



Pipeline initialized on device :  cuda


In [4]:
import torch

def get_class_labels(prompts:list[str]) -> torch.Tensor:
    class_labels = []
    for prompt in prompts:
        if prompt in vocab:
            class_labels.append(vocab[prompt])
        else:
            class_labels.append(-1)  # Use -1 for unknown classes
    return torch.tensor(class_labels, dtype=torch.long)


for batch in train_dataloader:
    input_images = batch['input']
    target_images = batch['target']
    prompts = batch['prompt']
    labels = get_class_labels(prompts).unsqueeze(1)  # Add a dimension for class labels
    print(labels.shape)
    outputs = pipeline(input_images=input_images,
                       class_labels=labels)
    
    break  # Just run one batch for demonstration purposes

torch.Size([16, 1])


  8%|▊         | 4/50 [00:48<09:13, 12.03s/it]


KeyboardInterrupt: 